In [36]:
import pandas as pd 
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [38]:
df=pd.read_csv("D:\pyt\Assignment\Assignment 13-KNN\Zoo.csv")

In [39]:
df.head()

,animal name,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,type
0,aardvark,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
1,antelope,1,0,0,1,0,0,0,1,1,1,0,0,4,1,0,1,1
2,bass,0,0,1,0,0,1,1,1,1,0,0,1,0,1,0,0,4
3,bear,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
4,boar,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,1


In [40]:
df.shape

(101, 18)

In [41]:
df.type.value_counts().sort_index()

1    41
2    20
3     5
4    13
5     4
6     8
7    10
Name: type, dtype: int64

In [42]:
df.isna().sum()

animal name    0
hair           0
feathers       0
eggs           0
milk           0
airborne       0
aquatic        0
predator       0
toothed        0
backbone       0
breathes       0
venomous       0
fins           0
legs           0
tail           0
domestic       0
catsize        0
type           0
dtype: int64

In [43]:
df.isnull().sum()

animal name    0
hair           0
feathers       0
eggs           0
milk           0
airborne       0
aquatic        0
predator       0
toothed        0
backbone       0
breathes       0
venomous       0
fins           0
legs           0
tail           0
domestic       0
catsize        0
type           0
dtype: int64

In [44]:
x= df.iloc[ : , 1:-1]
x

,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize
0,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1
1,1,0,0,1,0,0,0,1,1,1,0,0,4,1,0,1
2,0,0,1,0,0,1,1,1,1,0,0,1,0,1,0,0
3,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1
4,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,1,0,0,1,0,0,0,1,1,1,0,0,2,1,0,1
97,1,0,1,0,1,0,0,0,0,1,1,0,6,0,0,0
98,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1
99,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0


In [45]:
y = df.iloc[ : , -1]
y

0      1
1      1
2      4
3      1
4      1
      ..
96     1
97     6
98     1
99     7
100    2
Name: type, Length: 101, dtype: int64

# Applying KNN without hyper parameter tuning(without Gridsearch cv)

In [46]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler , MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

In [47]:
# Applying Standard Scalar to data
scaled = StandardScaler()
std_df = scaled.fit_transform(x)

In [48]:
x_train, x_test, y_train, y_test = train_test_split(std_df, y, test_size=0.2, random_state=42)

In [49]:
knn= KNeighborsClassifier(n_neighbors=4)

In [50]:
knn.fit(x_train, y_train)

KNeighborsClassifier(n_neighbors=4)

In [51]:
y_pred = knn.predict(x_test)

In [52]:
accuracy_score(y_test, y_pred)

0.9523809523809523

In [53]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00        12
           2       1.00      1.00      1.00         2
           3       0.00      0.00      0.00         1
           4       0.67      1.00      0.80         2
           6       1.00      1.00      1.00         3
           7       1.00      1.00      1.00         1

    accuracy                           0.95        21
   macro avg       0.78      0.83      0.80        21
weighted avg       0.92      0.95      0.93        21



# Applying KNN with hyper parameter tuning(using Gridsearch cv)

In [54]:
from sklearn.model_selection import GridSearchCV

In [55]:
neibhors= np.array(range(1,15))
parameters= [{"n_neighbors":neibhors ,"algorithm":['auto', 'ball_tree', 'kd_tree', 'brute'],
              "weights":['uniform', 'distance']}]


grid_search= GridSearchCV(estimator=knn, param_grid=parameters, scoring="accuracy",cv=3, n_jobs=-1 )

grid_search.fit(x_train, y_train)

GridSearchCV(cv=3, estimator=KNeighborsClassifier(n_neighbors=4), n_jobs=-1,
             param_grid=[{'algorithm': ['auto', 'ball_tree', 'kd_tree',
                                        'brute'],
                          'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
                          'weights': ['uniform', 'distance']}],
             scoring='accuracy')

In [56]:
grid_search.best_params_ 

{'algorithm': 'auto', 'n_neighbors': 3, 'weights': 'distance'}

In [57]:
# Applying above best parameters in model to get best accuracy
knn1= KNeighborsClassifier(algorithm='auto', n_neighbors= 3, weights='distance')

In [58]:
knn1.fit(x_train, y_train)

KNeighborsClassifier(n_neighbors=3, weights='distance')

In [59]:
y_pred1 = knn1.predict(x_test)

In [60]:
accuracy = np.round(accuracy_score(y_test, y_pred1)*100, 2)
print(accuracy)

100.0


In [61]:
print(classification_report(y_test, y_pred1))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00        12
           2       1.00      1.00      1.00         2
           3       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         2
           6       1.00      1.00      1.00         3
           7       1.00      1.00      1.00         1

    accuracy                           1.00        21
   macro avg       1.00      1.00      1.00        21
weighted avg       1.00      1.00      1.00        21



In [62]:
confusion_matrix(y_test, y_pred1)

array([[12,  0,  0,  0,  0,  0],
       [ 0,  2,  0,  0,  0,  0],
       [ 0,  0,  1,  0,  0,  0],
       [ 0,  0,  0,  2,  0,  0],
       [ 0,  0,  0,  0,  3,  0],
       [ 0,  0,  0,  0,  0,  1]], dtype=int64)